**Final Project for Algorithm Course**

**Enhancing Fleet Data Efficiency through Sorting Algorithms – Bubble Sort, Merge Sort, and Quick Sort**

**1. Imports**

In [1]:
import time
import random
import pandas as pd
import numpy as np
from tabulate import tabulate

**Sorting Algorithm Implementations**

In [2]:
def bubble_sort(arr):
    """Bubble Sort with comparison count."""
    n = len(arr)
    comparisons = 0
    arr = arr.copy()
    for i in range(n):
        for j in range(0, n-i-1):
            comparisons += 1
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return comparisons, arr

def merge_sort(arr):
    """Merge Sort with comparison count."""
    def _merge_sort(arr):
        if len(arr) <= 1:
            return 0, arr
        mid = len(arr) // 2
        left_comp, left = _merge_sort(arr[:mid])
        right_comp, right = _merge_sort(arr[mid:])
        comp = left_comp + right_comp
        merged = []
        i = j = 0
        while i < len(left) and j < len(right):
            comp += 1
            if left[i] < right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                j += 1
        merged.extend(left[i:])
        merged.extend(right[j:])
        return comp, merged
    return _merge_sort(arr.copy())

def quick_sort(arr):
    """Quick Sort with comparison count."""
    def _quick_sort(arr):
        if len(arr) <= 1:
            return 0, arr
        pivot = arr[random.randint(0, len(arr)-1)]
        less = [x for x in arr if x < pivot]
        equal = [x for x in arr if x == pivot]
        greater = [x for x in arr if x > pivot]
        left_comp, less_sorted = _quick_sort(less)
        right_comp, greater_sorted = _quick_sort(greater)
        comp = left_comp + right_comp + len(arr) - 1
        return comp, less_sorted + equal + greater_sorted
    return _quick_sort(arr.copy())

**Complexity and Scalability Calculation**

In [3]:
def calculate_space_complexity(algo_name, n):
    """Calculate theoretical space complexity."""
    if algo_name == "Bubble Sort":
        return "O(1)"
    elif algo_name == "Merge Sort":
        return f"O({n})"
    elif algo_name == "Quick Sort":
        return f"O({int(np.log2(n))})"
    return "N/A"

def get_scalability(algo_name):
    """Return scalability label."""
    if algo_name == "Bubble Sort":
        return "Low"
    else:
        return "High"

**Dataset Loading and Column Selection**

In [4]:
def load_datasets():
    """
    Load the four CSV datasets.
    Replace the file paths with the actual locations of your CSV files.
    """
    datasets = {
        "Fuel Data": pd.read_csv("fuel_data.csv"),
        "Maintenance Data": pd.read_csv("maintenance_data.csv"),
        "Driver Behavior Data": pd.read_csv("driver_behavior_data.csv"),
        "Location Data": pd.read_csv("location_data.csv")
    }
    return datasets

def get_sort_column(dataset_name, df):
    """
    Choose a sortable column for each dataset.
    Adjust these column names based on your actual CSV structure!
    """
    columns = {
        "Fuel Data": "engine_size",
        "Maintenance Data": "service_cost",
        "Driver Behavior Data": "acceleration",
        "Location Data": "dispatch_number"
    }
    col = columns.get(dataset_name)
    if col not in df.columns:
        for c in df.columns:
            if pd.api.types.is_numeric_dtype(df[c]):
                return c
        raise ValueError(f"No numeric column found in {dataset_name}")
    return col

**Datasaet Preprocessing**

In [9]:
def preprocess_data(df):
    """Clean and normalize dataset columns"""
    df = df.dropna() # Handling missing values here

    # Converting all numeric columns to float32 for consistency
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_cols] = df[numeric_cols].astype('float32')

    # Normalizing numeric columns (in the range of 0-1)
    df[numeric_cols] = (df[numeric_cols] - df[numeric_cols].min()) / (df[numeric_cols].max() - df[numeric_cols].min())

    return df

**Performance Metrics Table Prep**

In [12]:
def generate_performance_tables(datasets):
    """Generate formatted performance tables for all datasets (original style)"""
    column_names = ["Algorithm Case", "Time (s)", "Comparisons", "Space Complexity"]

    for data_name, df in datasets.items():
        col = get_sort_column(data_name, df)
        data = df[col].dropna().tolist()
        n = len(data)
        if n == 0:
            print(f"Warning: No data in {data_name} for column {col}")
            continue

        table_rows = []

        #Bubble Sort Cases
        start_time = time.time()
        comp, _ = bubble_sort(data.copy())
        exec_time = time.time() - start_time
        table_rows.append([
            "Bubble Sort - Best Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Bubble Sort", n)
        ])

        start_time = time.time()
        comp, _ = bubble_sort(data[::-1])
        exec_time = time.time() - start_time
        table_rows.append([
            "Bubble Sort - Worst Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Bubble Sort", n)
        ])

        # Merge Sort Cases
        start_time = time.time()
        comp, _ = merge_sort(data.copy())
        exec_time = time.time() - start_time
        table_rows.append([
            "Merge Sort - Best Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Merge Sort", n)
        ])

        start_time = time.time()
        comp, _ = merge_sort(data[::-1])
        exec_time = time.time() - start_time
        table_rows.append([
            "Merge Sort - Worst Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Merge Sort", n)
        ])

        start_time = time.time()
        comp, _ = merge_sort(random.sample(data, len(data)))
        exec_time = time.time() - start_time
        table_rows.append([
            "Merge Sort - Average Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Merge Sort", n)
        ])

        # Quick Sort Cases
        start_time = time.time()
        comp, _ = quick_sort(data.copy())
        exec_time = time.time() - start_time
        table_rows.append([
            "Quick Sort - Best Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Quick Sort", n)
        ])

        start_time = time.time()
        comp, _ = quick_sort(data[::-1])
        exec_time = time.time() - start_time
        table_rows.append([
            "Quick Sort - Worst Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Quick Sort", n)
        ])

        start_time = time.time()
        comp, _ = quick_sort(random.sample(data, len(data)))
        exec_time = time.time() - start_time
        table_rows.append([
            "Quick Sort - Average Case",
            round(exec_time, 6),
            comp,
            calculate_space_complexity("Quick Sort", n)
        ])

        print(f"\n Performance Metrics for Fleet Data Sorting Algorithms of the {data_name} Dataset")
        print(tabulate(table_rows, headers=column_names, tablefmt="grid"))

**Performance Metrics Table Generation**

In [13]:
if __name__ == "__main__":
    try:
        datasets = load_datasets()

        if not datasets:
            print("Error: No datasets loaded. Check your CSV files exist in the current directory.")
            print("Expected files: fuel_data.csv, maintenance_data.csv, driver_behavior_data.csv, location_data.csv")
        else:
            generate_performance_tables(datasets)

    except Exception as e:
        print(f"An error occurred: {str(e)}")


 Performance Metrics for Fleet Data Sorting Algorithms of the Fuel Data Dataset
+---------------------------+------------+---------------+--------------------+
| Algorithm Case            |   Time (s) |   Comparisons | Space Complexity   |
+===========================+============+===============+====================+
| Bubble Sort - Best Case   |  25.4802   |     254375290 | O(1)               |
+---------------------------+------------+---------------+--------------------+
| Bubble Sort - Worst Case  |  48.271    |     254375290 | O(1)               |
+---------------------------+------------+---------------+--------------------+
| Merge Sort - Best Case    |   0.052047 |        193190 | O(22556)           |
+---------------------------+------------+---------------+--------------------+
| Merge Sort - Worst Case   |   0.044266 |        169260 | O(22556)           |
+---------------------------+------------+---------------+--------------------+
| Merge Sort - Average Case |   0.07850